<a href="https://colab.research.google.com/github/sagihaider/Biogas_Prediction/blob/master/MLP_MISO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/sagihaider/Biogas_Prediction.git 

import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dense
from sklearn.preprocessing import normalize
from keras import optimizers
from sklearn.model_selection import train_test_split

Cloning into 'Biogas_Prediction'...
remote: Enumerating objects: 339, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 339 (delta 8), reused 46 (delta 1), pack-reused 275
Receiving objects: 100% (339/339), 72.42 MiB | 36.59 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [2]:
data = pd.read_excel('/content/Biogas_Prediction/Data/Gasification Data_Daya.xlsx', index_col=0, header=0)
data = data.iloc[1:223,:14] 
display(list(data.columns.values))

['C [%wt db]',
 'H [%wt db]',
 'O [%wt db]',
 'Moisture [%wt]',
 'Ash [%wt db]',
 'ER [-]',
 'T [ºC]',
 'Bed material',
 'Steam/Biomass',
 'H2 [%vol N2 free]',
 'CO [%vol N2 free]',
 'CO2 [%vol N2 free]',
 'CH4 [%vol N2 free]',
 'GY [Nm3/kg daf]']

In [3]:
# Drop null balues and store dataframe in dataframe 2
data=data.dropna()
print(data)

#Check Null values again after removing
print(data.isnull().values.any())
print(data.isna().values.any())

X_all = data.iloc[:, :9]
y_all = data.iloc[:, 9:]

        C [%wt db] H [%wt db]  ...  CH4 [%vol N2 free] GY [Nm3/kg daf]
Sr No.                         ...                                    
2            50.00        5.7  ...            6.474820        2.500000
3            50.00        5.7  ...            7.164179        2.500000
4            50.00        5.7  ...            6.474820        2.100000
5            50.00        5.7  ...            6.474820        2.400000
6            50.00        5.7  ...            8.181818        2.100000
...            ...        ...  ...                 ...             ...
219          46.85        6.3  ...            9.706147        0.947532
220          46.85        6.3  ...            9.455065        0.975752
221          46.76       5.68  ...            9.933682        0.971784
222          46.76       5.68  ...            8.778402        0.982989
223          46.76       5.68  ...            8.534014        0.973816

[187 rows x 14 columns]
False
False


In [4]:

input_columns = list(X_all.columns.values)
input_columns = [i.split(' [', 1)[0] for i in input_columns]
output_columns = list(y_all.columns.values)
output_columns = [i.split(' [', 1)[0] for i in output_columns]
print(input_columns, output_columns)

Xnorm = normalize(X_all, 'l2', axis=1)
ynorm = normalize(y_all, 'l2', axis=1)

['C', 'H', 'O', 'Moisture', 'Ash', 'ER', 'T', 'Bed material', 'Steam/Biomass'] ['H2', 'CO', 'CO2', 'CH4', 'GY']


In [5]:
from keras import backend as K

def coeff_determination(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [ ]:
# Create MISO using train & test split without R^2
nrow, ncol=ynorm.shape

for i in range(ncol):

  X = Xnorm
  y = ynorm[:,i]

  X_train, X_test, y_train, y_test = train_test_split(Xnorm, y, test_size=0.33)

  # define the keras model
  model = Sequential()
  model.add(Dense(10, input_shape = (9,), activation = 'sigmoid'))
  model.add(Dense(10, activation = 'sigmoid'))
  model.add(Dense(10, activation = 'sigmoid'))
  model.add(Dense(1))
  model.summary()

  sgd = optimizers.SGD(lr = 0.01)    # stochastic gradient descent optimizer

  model.compile(optimizer = sgd, loss = 'mean_squared_error', metrics = ['mse']) # for regression problems, mean squared error (MSE) is often employed
  history=model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size = 25, epochs = 20, verbose = 1)

  # results = model.evaluate(X_test, y_test)
  # print('loss: ', results[0])
  # print('mse: ', results[1])

  import matplotlib.pyplot as plt
  print(history.history.keys())
  # list all data in history
  print(history.history.keys())
  # summarize history for accuracy
  plt.plot(history.history['mse'])
  plt.plot(history.history['val_mse'])
  plt.title(output_columns[i])
  plt.ylabel('MSE')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()
  del model


In [22]:
# Create MISO using train & test split without R^2
from sklearn.metrics import mean_squared_error

rsqaure_list = []
mse_list = []
nrow, ncol=ynorm.shape

for i in range(ncol):

  X = Xnorm
  y = ynorm[:,i]

  X_train, X_test, y_train, y_test = train_test_split(Xnorm, y, test_size=0.33)

  # define the keras model
  model = Sequential()
  model.add(Dense(10, input_shape = (9,), activation = 'sigmoid'))
  # model.add(Dense(10, activation = 'sigmoid'))
  model.add(Dense(10, activation = 'sigmoid'))
  model.add(Dense(1))
  model.summary()

  sgd = optimizers.SGD(lr = 0.01)    # stochastic gradient descent optimizer

  model.compile(optimizer = sgd, loss = 'mean_squared_error', metrics = ['mse']) # for regression problems, mean squared error (MSE) is often employed
  history=model.fit(X_train, y_train, batch_size = 25, epochs = 20, verbose = 1)
  
  y_pred = model.predict(X_test)
  y_pred = y_pred.ravel()
  r2=r2_score(y_test, y_pred)
  mse_val=mean_squared_error(y_test, y_pred)

  print('The RSquare Value is:', r2)
  rsqaure_list.append(r2)

  print('The mse Value is:', mse_val)
  mse_list.append(mse_val)

print(rsqaure_list)
print(mse_list)
  

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_124 (Dense)            (None, 10)                100       
_________________________________________________________________
dense_125 (Dense)            (None, 10)                110       
_________________________________________________________________
dense_126 (Dense)            (None, 1)                 11        
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
5/5 [==============================] - 0s 2ms/step - loss: 0.0546 - mse: 0.0546
Epoch 2/20
5/5 [==============================] - 0s 2ms/step - loss: 0.0375 - mse: 0.0375
Epoch 3/20
5/5 [==============================] - 0s 2ms/step - loss: 0.0305 - mse: 0.0305
Epoch 4/20
5/5 [==============================] - 0s 3ms/step - loss: 0.0278 - mse: 0.0278
Epoch 5/20
5/5 [=====